In [27]:
import os
import numpy as np
from scipy import signal
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [28]:
WRITE_IMAGES = False
WRITE_TABLE = False
INIT = "t"
DEST_COORDINATES = [0, 0, 1]


SAVE_PATH = f"../results_plots"

In [29]:
os.makedirs(SAVE_PATH, exist_ok=True)

In [30]:
models = {
    "b_dist-lstm-v-61": {"path" : f"../results-nav-b_dist-lstm-v-61/plt/i{INIT}/", "color": "purple"},
    # "dist-err-rew-wi-v-50": {"path" : f"../results-nav-dist-err-rew-w_i-v-50/plt/i{INIT}/", "color": "red"},
    "b_dist-err-rew-wi-v-51": {"path" : f"../results-nav-b_dist-err-rew-w_i-v-51/plt/i{INIT}/", "color": "orange"},
    # "dist-v-3": {"path" : f"../results-nav-dist-v-3/plt/i{INIT}/", "color": "blue"},
    "b_dist-v-31": {"path" : f"../results-nav-b_dist-v-31/plt/i{INIT}/", "color": "blue"},
    "v-21": {"path" : f"../results-nav-v-21/plt/i{INIT}/", "color": "green"},
}

In [31]:
def plot(dir, dist):
    fig_traj = px.line_3d()
    fig_traj.update_layout(
        title_text=f"Trajectories {dir=} {dist=}",
        # scene=dict(
        #     xaxis=dict(range=[-1, 1]),
        #     yaxis=dict(range=[-1, 1]),
        #     zaxis=dict(range=[0, 2]),
        # ),
    )

    fig_action_mag = px.scatter()
    fig_action_mag.update_layout(
        title_text=f"Error ||action|| - ||executed|| (ideal 0) {dir=} {dist=}",
    )

    ret = {
        "dir": dir,
        "dist": dist,
        # "distance_travelled": {},
        "smoothness": {},
        "avg_ascent_step": {},
        "steady_state_err": {},
        "converged": {},
    }

    for mdl in models:
        try:
            df = pd.read_csv(os.path.join(models[mdl]["path"], f"{dir}_{dist}.csv"))

            distance = 0
            for i in range(len(df) - 1):
                distance += np.linalg.norm(df.iloc[i][["x", "y", "z"]] - df.iloc[i + 1][["x", "y", "z"]])


            dt = df.iloc[:-1][["x", "y", "z"]].to_numpy() - df.iloc[1:][["x", "y", "z"]].to_numpy()

            # dt = dt / np.expand_dims(np.linalg.norm(dt, axis=1), axis=1)
            ddt = dt[:-1] - dt[1:]

            # ddt = np.diff(np.diff(df.iloc[:-1][["x", "y", "z"]].to_numpy(), axis=0), axis=0)

            ddt_mag = np.linalg.norm(ddt, axis=1) ** 2

            smoothness = np.sum(ddt_mag)

            ### average stride during ascent (consider 5 steps)
            dt_ascent = dt[:5, :]

            avg_ascent_step = np.mean(np.linalg.norm(dt_ascent, axis=1))

            fig_traj.add_scatter3d(
                x=df.x,
                y=df.y,
                z=df.z,
                line=dict(color=models[mdl]["color"]),
                marker=dict(size=3),
                name=mdl,
            )

            fig_action_mag.add_scatter(
                x=df.index[:10],
                y=signal.savgol_filter(df.action_mag[:-1] - np.linalg.norm(dt, axis=1), 20, 2)[:10],
                line=dict(color=models[mdl]["color"]),
                marker=dict(size=3),
                name=mdl,
            )

            # ret["distance_travelled"][mdl] = distance

            ret["smoothness"][mdl] = smoothness

            ret["avg_ascent_step"][mdl] = avg_ascent_step

            ret["steady_state_err"][mdl] = np.linalg.norm(df.iloc[-1][["x", "y", "z"]] - DEST_COORDINATES)

            ret["converged"][mdl] = (
                "Yes" if np.linalg.norm(df.iloc[-1][["x", "y", "z"]] - DEST_COORDINATES) < 0.20 else "No"
            )

            # ret[f"distance_travelled.{mdl}"] = dt
            # ret[f"convereged.{mdl}"] = "Yes" if np.linalg.norm(df.iloc[-1, :] - [0, 0, 1]) < 0.1 else "No"
        except (IndexError, KeyError, AttributeError) as e:
            print(f"skipping: {mdl}: {e}")
            continue

    fig_traj.show()
    fig_action_mag.show()

    if WRITE_IMAGES:
        fig_traj.write_html(os.path.join(SAVE_PATH, f"{dir}-{dist}.html"))
        fig_traj.write_image(os.path.join(SAVE_PATH, f"traj_{dir}-{dist}.png"))
        fig_traj.write_image(os.path.join(SAVE_PATH, f"error_{dir}-{dist}.png"))

    return ret

In [32]:
files = os.listdir(models[list(models.keys())[0]]["path"])
files = list(filter(lambda name: name.startswith("x") or name.startswith("y") or name.startswith("z") or name.startswith("xyz"), files))

results = []

files_df = []

for file in files:
    name = file.split(".csv")[0]
    (dir, dist) = name.split("_")

    files_df.append({
        "dir": dir,
        "dist": dist,
    })

files_df = pd.DataFrame(files_df)

files_df.sort_values(by=["dir", "dist"], inplace=True)

for index, row in files_df.iterrows():
    ret = plot(row["dir"], row["dist"])
    results.append(ret)

skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


...

skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


skipping: b_dist-lstm-v-61: 'DataFrame' object has no attribute 'action_mag'


In [33]:
results_df = pd.json_normalize(results)

results_df.sort_values(by=["dir", "dist"], inplace=True)

results_df.reset_index(inplace=True)

results_df_cols = results_df.columns
print(results_df_cols)

Index(['index', 'dir', 'dist', 'smoothness.b_dist-err-rew-wi-v-51',
       'smoothness.b_dist-v-31', 'smoothness.v-21',
       'avg_ascent_step.b_dist-err-rew-wi-v-51', 'avg_ascent_step.b_dist-v-31',
       'avg_ascent_step.v-21', 'steady_state_err.b_dist-err-rew-wi-v-51',
       'steady_state_err.b_dist-v-31', 'steady_state_err.v-21',
       'converged.b_dist-err-rew-wi-v-51', 'converged.b_dist-v-31',
       'converged.v-21'],
      dtype='object')


In [34]:
def color_conv(value):
    color = "#ed7b7b" if value == "No" else ""
    return f"background-color: {color}"


In [35]:
def color_min_metric_group(temp_cols):
    # temp_cols = list(filter(lambda col_name: col_name.startswith(col_name), all_cols))

    def color_func(row):
        row_vals = row[temp_cols]
        is_min = row_vals == row_vals.min()

        return ['background-color: #95ed8e' if v else '' for v in is_min]
    
    return color_func

In [36]:
# color conv
temp_cols_conv = list(filter(lambda col_name: col_name.startswith("converged"), results_df_cols))

# color smooth
temp_cols_sm = list(filter(lambda col_name: col_name.startswith("smoothness"), results_df_cols))

# color avg_ascent_step
temp_cols_as = list(filter(lambda col_name: col_name.startswith("avg_ascent_step"), results_df_cols))

# color ss
temp_cols_ss = list(filter(lambda col_name: col_name.startswith("steady_state_err"), results_df_cols))

# color dis
temp_cols_dis = list(filter(lambda col_name: col_name.startswith("distance_travelled"), results_df_cols))

results_color_df = results_df.style.applymap(color_conv, subset=temp_cols_conv) \
    .apply(color_min_metric_group(temp_cols_sm), subset=temp_cols_sm, axis=1) \
    .apply(color_min_metric_group(temp_cols_as), subset=temp_cols_as, axis=1) \
    .apply(color_min_metric_group(temp_cols_ss), subset=temp_cols_ss, axis=1) \
    .apply(color_min_metric_group(temp_cols_dis), subset=temp_cols_dis, axis=1) \
    .set_properties(**{'border-color': 'black', 'border-width': '1px', 'border': '1px solid black', 'border-collapse': 'collapse', 'text-align': 'center'})

In [37]:
# temp_cols = list(filter(lambda col_name: col_name.startswith("steady_state_err"), results_df_cols))
# results_df = results_df.style.highlight_min(color="green", axis=1, subset=temp_cols)

# temp_cols = list(filter(lambda col_name: col_name.startswith("smoothness"), results_df_cols))
# results_df = results_df.style.highlight_min(color="green", axis=1, subset=temp_cols)

In [38]:
# cols = pd.MultiIndex.from_tuples([
#     ("dir", ""),
#     ("dist", ""),
#     ("distance_travelled", "err"),
#     ("distance_travelled", "no-err"),
#     ("converged", "err"),
#     ("converged", "no-err"),
# ])

# results_df = pd.json_normalize(results)

# pd.DataFrame(results_df.values, columns=cols)

In [39]:
if WRITE_TABLE:
    results_color_df.to_html(os.path.join(SAVE_PATH, f"results.html"), index=False)